In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
import random
import ipywidgets as widgets
from IPython.display import display

# Initialize an empty task list
tasks = pd.DataFrame(columns=['description', 'priority'])

# Load pre-existing tasks from a CSV file (if any)
try:
    tasks = pd.read_csv('tasks.csv')
except FileNotFoundError:
    pass

# Function to save tasks to a CSV file
def save_tasks():
    tasks.to_csv('tasks.csv', index=False)

# Function to train the task priority classifier
def train_classifier():
    vectorizer = CountVectorizer()
    clf = MultinomialNB()
    model = make_pipeline(vectorizer, clf)
    model.fit(tasks['description'], tasks['priority'])
    return model

# Function to add a task to the list
def add_task(description, priority):
    global tasks  # Declare tasks as a global variable
    new_task = pd.DataFrame({'description': [description], 'priority': [priority]})
    tasks = pd.concat([tasks, new_task], ignore_index=True)
    save_tasks()

# Function to remove a task by description
def remove_task(description):
    global tasks  # Declare tasks as a global variable
    tasks = tasks[tasks['description'] != description]
    save_tasks()

# Function to list all tasks
def list_tasks():
    if tasks.empty:
        return "No tasks available."
    else:
        return tasks.to_string(index=False)

# Function to recommend a task based on machine learning
def recommend_task(model):
    if not tasks.empty:
        high_priority_tasks = tasks[tasks['priority'] == 'High']

        if not high_priority_tasks.empty:
            random_task = random.choice(high_priority_tasks['description'].values)
            return f"Recommended task: {random_task} - Priority: High"
        else:
            return "No high-priority tasks available for recommendation."
    else:
        return "No tasks available for recommendations."

# Create interactive widgets
description_input = widgets.Text(description="Task description:")
priority_dropdown = widgets.Dropdown(options=['Low', 'Medium', 'High'], description="Priority:")
add_button = widgets.Button(description="Add Task")
remove_input = widgets.Text(description="Task to remove:")
remove_button = widgets.Button(description="Remove Task")
list_button = widgets.Button(description="List Tasks")
recommend_button = widgets.Button(description="Recommend Task")
output = widgets.Output()

# Define event handlers
def add_button_click(b):
    add_task(description_input.value, priority_dropdown.value)
    with output:
        print("Task added successfully.")

def remove_button_click(b):
    remove_task(remove_input.value)
    with output:
        print("Task removed successfully.")

def list_button_click(b):
    with output:
        print(list_tasks())

def recommend_button_click(b):
    model = train_classifier()
    with output:
        print(recommend_task(model))

# Attach event handlers
add_button.on_click(add_button_click)
remove_button.on_click(remove_button_click)
list_button.on_click(list_button_click)
recommend_button.on_click(recommend_button_click)

# Display widgets
display(description_input, priority_dropdown, add_button)
display(remove_input, remove_button)
display(list_button, recommend_button)
display(output)


Text(value='', description='Task description:')

Dropdown(description='Priority:', options=('Low', 'Medium', 'High'), value='Low')

Button(description='Add Task', style=ButtonStyle())

Text(value='', description='Task to remove:')

Button(description='Remove Task', style=ButtonStyle())

Button(description='List Tasks', style=ButtonStyle())

Button(description='Recommend Task', style=ButtonStyle())

Output()